
# Pytorch + HuggingFace 
## KoElectra Model
박장원님의 KoElectra-small 사용<br>
https://monologg.kr/2020/05/02/koelectra-part1/<br>
https://github.com/monologg/KoELECTRA

## Dataset
네이버 영화 리뷰 데이터셋<br>
https://github.com/e9t/nsmc

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
꼭 GPU로 해주세요 - 1epoch 당 약 20분 소요

In [1]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
#!pip install transformers


In [3]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
import gc
from sklearn.model_selection import KFold
import numpy as np
import os

In [5]:
# GPU 사용
device = torch.device("cuda")
MODEL_NAME = "monologg/koelectra-small-v3-discriminator"
pre_MODEL_NAME = "result/MODEL_KoELECTRA-Small-v3-discriminator__3.pt"

In [45]:
#naver movie, Hotel, shopping 
class NHSDataset(Dataset): 
  
    def __init__(self, csv_file):
        # NaN값 제거...
        self.dataset = pd.read_csv(csv_file, sep='\t').dropna(axis=0) 
        # 중복제거
        #self.dataset.drop_duplicates(subset=['text'], inplace=True)
        
        print(self.dataset)
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        
  
    def __len__(self):
        return len(self.dataset)
  
    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, :].values
        text = row[0]
        #y = row[1]

        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            truncation=True,
            max_length=256,
            pad_to_max_length=True,
            add_special_tokens=True
            )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, text

In [46]:
test_dataset = NHSDataset('data/key_pre/small_test.txt')

                                                  text
0                    친구들하고 오랜만에 급 계획으로 예약하여 호캉스 다녀왔습니다
1                  호텔 주변에 음식점도 많고 부대시설도 잘 되어 있어서 편햇습니다
2                         그리고 바로 역도 가까이 있어서 이동이 편하더라구요
3                              그런데 객실이 좀 많이 낙후되어 있더라구요
4                         그 점을 빼곤 서비스 전체 모든 면에서 괜찮았습니다
..                                                 ...
904                         화장실도 놀랍고 전반적으로 그냥 모텔 수준입니다
905                          장점 위치적인 요인이 가장 플러스로 작용합니다
906                                  단점 편하지 않은 직원들의 응대
907  두 번 다시는 안 가요 복도에서는 삼겹살 냄새나고 방음도 잘 안 되서 옆 방 초인종...
908                                      위치 친절한 직원 사우나

[909 rows x 1 columns]


In [48]:
test_dataset[0]

e:\build\project\programmers\4_party_project\env\lib\site-packages\transformers\tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


(tensor([    2,  6554,  4006,  4279,  4219, 10953,  4073,  2132,  6333, 10749,
          8866, 26509,  3802, 28693, 14991,  4576,  6216,     3,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [32]:

test_dataset = NHSDataset('data/key_pre/small_test.txt')
def init():
    
    
def model_setting():
    model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME).to(device)

In [42]:
#train데이터 test데이터 NHSDataset class에 넣어줌
#train_dataset = NHSDataset('data/sentiment_test_dataset/hotel_movie_train.txt')
print(test_dataset[0])

(tensor([    2,  6554,  4006,  4279,  4219, 10953,  4073,  2132,  6333, 10749,
         8866, 26509,  3802, 28693, 14991,  4576,  6216,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,

# Create Model

In [34]:


# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

# Model 저장되있는거 돌려보기

In [8]:

# model.load_state_dict(torch.load("원하는모델.pt"))
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))


In [9]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

# Learn

In [35]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [36]:
epochs = 10
batch_size = 8

In [37]:
optimizer = AdamW(model.parameters(), lr=1e-5)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [9]:
losses = []
accuracies = []

for i in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch, original_text in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
            print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
    losses.append(total_loss)
    accuracies.append(correct.float() / total)
    print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/12944 [00:00<?, ?it/s]

e:\build\project\programmers\4_party_project\env\lib\site-packages\transformers\tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 67.12117129564285 Accuracy: tensor(0.6000, device='cuda:0')
Batch Loss: 117.15007968246937 Accuracy: tensor(0.7025, device='cuda:0')
Batch Loss: 158.89970202744007 Accuracy: tensor(0.7450, device='cuda:0')
Batch Loss: 196.61772364377975 Accuracy: tensor(0.7709, device='cuda:0')
Batch Loss: 236.1447340324521 Accuracy: tensor(0.7835, device='cuda:0')
Batch Loss: 276.7157583460212 Accuracy: tensor(0.7912, device='cuda:0')
Batch Loss: 313.0681369677186 Accuracy: tensor(0.7993, device='cuda:0')
Batch Loss: 351.6782390102744 Accuracy: tensor(0.8034, device='cuda:0')
Batch Loss: 386.4174063280225 Accuracy: tensor(0.8092, device='cuda:0')
Batch Loss: 416.29794661700726 Accuracy: tensor(0.8163, device='cuda:0')
Batch Loss: 452.4189935140312 Accuracy: tensor(0.8193, device='cuda:0')
Batch Loss: 484.3220498301089 Accuracy: tensor(0.8235, device='cuda:0')
Batch Loss: 519.0525521375239 Accuracy: tensor(0.8256, device='cuda:0')
Batch Loss: 552.440062534064 Accuracy: tensor(0.8284, device

Batch Loss: 5996.049318337813 Accuracy: tensor(0.6729, device='cuda:0')
Batch Loss: 6065.499453449622 Accuracy: tensor(0.6716, device='cuda:0')
Batch Loss: 6134.885305786505 Accuracy: tensor(0.6701, device='cuda:0')
Batch Loss: 6204.286832058802 Accuracy: tensor(0.6687, device='cuda:0')
Batch Loss: 6273.840327048674 Accuracy: tensor(0.6672, device='cuda:0')
Batch Loss: 6343.508880281821 Accuracy: tensor(0.6656, device='cuda:0')
Batch Loss: 6413.13424625434 Accuracy: tensor(0.6641, device='cuda:0')
Batch Loss: 6482.46738162078 Accuracy: tensor(0.6628, device='cuda:0')
Batch Loss: 6551.94710620679 Accuracy: tensor(0.6615, device='cuda:0')
Batch Loss: 6621.31428267993 Accuracy: tensor(0.6604, device='cuda:0')
Batch Loss: 6690.639566982165 Accuracy: tensor(0.6593, device='cuda:0')
Batch Loss: 6759.999844217673 Accuracy: tensor(0.6580, device='cuda:0')
Batch Loss: 6829.232450867072 Accuracy: tensor(0.6569, device='cuda:0')
Batch Loss: 6898.6151551846415 Accuracy: tensor(0.6556, device='cuda

  0%|          | 0/12944 [00:00<?, ?it/s]

...

Batch Loss: 5.852655531605706 Accuracy: tensor(0.9800, device='cuda:0')
Batch Loss: 13.71093392977491 Accuracy: tensor(0.9769, device='cuda:0')
Batch Loss: 18.939179955865256 Accuracy: tensor(0.9792, device='cuda:0')
Batch Loss: 23.183268115390092 Accuracy: tensor(0.9812, device='cuda:0')
Batch Loss: 27.57419253757689 Accuracy: tensor(0.9818, device='cuda:0')
Batch Loss: 32.856477559544146 Accuracy: tensor(0.9815, device='cuda:0')
Batch Loss: 41.143200217047706 Accuracy: tensor(0.9805, device='cuda:0')
Batch Loss: 45.86200051987544 Accuracy: tensor(0.9809, device='cuda:0')
Batch Loss: 50.19208702258766 Accuracy: tensor(0.9815, device='cuda:0')
Batch Loss: 56.018194254953414 Accuracy: tensor(0.9813, device='cuda:0')
Batch Loss: 60.8090911176987 Accuracy: tensor(0.9819, device='cuda:0')
Batch Loss: 67.74303884082474 Accuracy: tensor(0.9817, device='cuda:0')
Batch Loss: 74.62837603269145 Accuracy: tensor(0.9815, device='cuda:0')
Batch Loss: 79.20470863941591 Accuracy: tensor(0.9818, devic

Batch Loss: 709.6843005364644 Accuracy: tensor(0.9804, device='cuda:0')
Batch Loss: 716.3702172573539 Accuracy: tensor(0.9804, device='cuda:0')
Batch Loss: 723.3104105949751 Accuracy: tensor(0.9804, device='cuda:0')
Batch Loss: 730.6427946821204 Accuracy: tensor(0.9804, device='cuda:0')
Batch Loss: 735.1909617237397 Accuracy: tensor(0.9804, device='cuda:0')
Batch Loss: 744.0410575460992 Accuracy: tensor(0.9803, device='cuda:0')
Batch Loss: 749.4889092170051 Accuracy: tensor(0.9803, device='cuda:0')
Batch Loss: 754.4830842196825 Accuracy: tensor(0.9804, device='cuda:0')
Batch Loss: 760.2359726301511 Accuracy: tensor(0.9804, device='cuda:0')
Batch Loss: 770.0348377879127 Accuracy: tensor(0.9803, device='cuda:0')
Batch Loss: 779.3866382280248 Accuracy: tensor(0.9803, device='cuda:0')
Batch Loss: 785.8561215835507 Accuracy: tensor(0.9802, device='cuda:0')
Batch Loss: 789.9500073363888 Accuracy: tensor(0.9803, device='cuda:0')
Batch Loss: 797.7232310465188 Accuracy: tensor(0.9802, device='c

  0%|          | 0/12944 [00:00<?, ?it/s]

Batch Loss: 5.058940358925611 Accuracy: tensor(0.9875, device='cuda:0')
Batch Loss: 10.276217825012282 Accuracy: tensor(0.9869, device='cuda:0')
Batch Loss: 15.239789778832346 Accuracy: tensor(0.9879, device='cuda:0')
Batch Loss: 16.712170756072737 Accuracy: tensor(0.9900, device='cuda:0')
Batch Loss: 20.892184171359986 Accuracy: tensor(0.9890, device='cuda:0')
Batch Loss: 26.03598566353321 Accuracy: tensor(0.9881, device='cuda:0')
Batch Loss: 29.030446673219558 Accuracy: tensor(0.9882, device='cuda:0')
Batch Loss: 32.76918498094892 Accuracy: tensor(0.9883, device='cuda:0')
Batch Loss: 37.152250258775894 Accuracy: tensor(0.9882, device='cuda:0')
Batch Loss: 39.682063989457674 Accuracy: tensor(0.9885, device='cuda:0')
Batch Loss: 47.35774250025861 Accuracy: tensor(0.9873, device='cuda:0')
Batch Loss: 55.579300177516416 Accuracy: tensor(0.9865, device='cuda:0')
Batch Loss: 64.62539831490722 Accuracy: tensor(0.9854, device='cuda:0')
Batch Loss: 68.95276023715269 Accuracy: tensor(0.9853, d

Batch Loss: 584.1821193032665 Accuracy: tensor(0.9843, device='cuda:0')
Batch Loss: 588.7689832437318 Accuracy: tensor(0.9844, device='cuda:0')
Batch Loss: 592.3958280609222 Accuracy: tensor(0.9844, device='cuda:0')
Batch Loss: 600.5609871849883 Accuracy: tensor(0.9843, device='cuda:0')
Batch Loss: 604.8430113089271 Accuracy: tensor(0.9844, device='cuda:0')
Batch Loss: 609.4872746473411 Accuracy: tensor(0.9844, device='cuda:0')
Batch Loss: 616.0418544050772 Accuracy: tensor(0.9843, device='cuda:0')
Batch Loss: 621.8254794834647 Accuracy: tensor(0.9843, device='cuda:0')
Batch Loss: 626.0420755945379 Accuracy: tensor(0.9843, device='cuda:0')
Batch Loss: 631.4305859507294 Accuracy: tensor(0.9843, device='cuda:0')
Batch Loss: 636.8815622739494 Accuracy: tensor(0.9842, device='cuda:0')
Batch Loss: 641.2389526104671 Accuracy: tensor(0.9843, device='cuda:0')
Batch Loss: 650.2011355275754 Accuracy: tensor(0.9842, device='cuda:0')
Batch Loss: 657.315684769419 Accuracy: tensor(0.9841, device='cu

In [38]:
model.load_state_dict(torch.load(pre_MODEL_NAME))

<All keys matched successfully>

# 테스트 데이터셋 정확도및 강한(긍정,부정) 확률 확인


In [39]:
model.eval()

test_correct = 0
test_total = 0
result = []
for input_ids_batch, attention_masks_batch, original_text in tqdm(test_loader):
    #y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    for idx in range(len(y_pred)):
        tmp = []
        tmp.append(original_text[idx])
        print(f'기존문장: {original_text[idx]}')
        pred_tensor = F.sigmoid(y_pred[idx])
        print(f'부정확률={round(float(pred_tensor[0]*100),2)}%, 긍정확률={round(float(pred_tensor[1]*100),2)}%')
        print("")
        
        tmp.append(round(float(pred_tensor[0]*100),2))
        tmp.append(round(float(pred_tensor[1]*100),2))
        result.append(tmp)
    _, predicted = torch.max(y_pred, 1)
    #test_correct += (predicted == y_batch).sum()
    #test_total += len(y_batch)

#print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/114 [00:00<?, ?it/s]

e:\build\project\programmers\4_party_project\env\lib\site-packages\transformers\tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
e:\build\project\programmers\4_party_project\env\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


기존문장: 전철역과 가깝고 호텔 부페는 저렴하고 종류도 많아 놀고 먹기는 딱입니다
부정확률=3.56%, 긍정확률=94.45%

기존문장: 12시 체크아웃 주말이면 투숙객도 무조건 발렛 강요함 발렛비 개인으로 내야 함 비싼 모텔 수준
부정확률=87.01%, 긍정확률=13.91%

기존문장: 방음이 잘 안 됐어요
부정확률=93.06%, 긍정확률=6.84%

기존문장: 너무하다 증 말
부정확률=72.48%, 긍정확률=28.2%

기존문장: 이 호텔을 예약한 경우 이미 변경할 수 있고 사용하도록 조심하세요
부정확률=90.04%, 긍정확률=10.03%

기존문장: 퇴실이 12시인데 9시에 차를 빼달라고 아니면 10분에 2천원 이라고 이건 어찌저찌 해경함
부정확률=79.11%, 긍정확률=21.79%

기존문장: 다음에 또 일이 생기면 여기를 찾게 될 것 같습니다
부정확률=9.84%, 긍정확률=87.43%

기존문장: 청결 상태가 최악입니다
부정확률=91.97%, 긍정확률=8.04%

기존문장: 좋은 점이 없습니다
부정확률=38.38%, 긍정확률=60.32%

기존문장: 외국 손님 오셔서 예약했는데 숙소 자체는 괜찮은데 클럽이 있어서 너무 시끄럽고 주변이 지저분하더라구 요 ㅠㅠ 내국인은 괜찮지만 외국인에게는 추천하고 싶지 않은 호텔입니다
부정확률=89.69%, 긍정확률=11.19%

기존문장: 이는 음 양성애자 단지 선호하는 딱딱한 침대 방은 조용하고 그래서 전혀 문제가 또한 와이파이 잘 들어 절곡돌기 안 좋았던 점 방은 더럽고 심각한 업데이트 전체 본 제한된 예산 이는 나쁘지 않은 선택 하지만 더 나은 선택이 많이 있다
부정확률=90.71%, 긍정확률=9.72%

기존문장: 이런 곳을 호텔이라고 부를 수 있을까 싶습니다 시트는 빨아도 더러운 건지 온통 정체를 알고 싶지도 않은 여러 종류의 얼룩들로 가득했고 벽에는 구멍까지 나 고 온갖 신발 자국들로 더러웠습니다
부정확률=89.96%, 긍정확률=10.64%

기존문장: 있을 거 다 있고 창 밖 한강 풍경도 멋있어요
부정확률

부정확률=90.65%, 긍정확률=9.59%

기존문장: 가격 대비 편안함
부정확률=6.11%, 긍정확률=91.46%

기존문장: 그리고 수영장과 사우나는 입실하는 날 결제하면 그날만 사용 가능하고 다음날 아침에 또 이용하려면 다시 결제해야 합니다
부정확률=47.53%, 긍정확률=53.22%

기존문장: 좀 더 쾌적한 환경이 필요합니다
부정확률=89.59%, 긍정확률=10.56%

기존문장: 무슨 호텔이 이럴까 호텔이라고 부를 수 없을 정도 피트니스 청소를 얼마나 하지 않았으면 1주일이 넘어도 컵이 그대로 있고 여기저기 먼지가 수북 락커에도 먼지가 목욕탕에는 스마트폰 사용하는 자들이 득실거리고 ㅎ 직원이 누구인지 리버 사이드 모텔로 이름 바꿔 라 호텔이라는 이름을 더럽히지 말고
부정확률=90.86%, 긍정확률=9.41%

기존문장: 덩치가 큰 편이라서 큰 침대를 요청했었는데 침대 두 개를 붙여줘서 잠을 잘 자지 못했습니다
부정확률=81.72%, 긍정확률=20.08%

기존문장: 위치만 좋아요
부정확률=5.59%, 긍정확률=92.16%

기존문장: 위치는 좋음
부정확률=4.68%, 긍정확률=93.13%

기존문장: 하지 않는 것을 알고 있는지 다른 일부 팬들은 추출 밖 조명을 작동하지 않습니다
부정확률=90.97%, 긍정확률=9.11%

기존문장: 무엇보다 패밀리 레스토랑 가격과 별 차이가 없는데 훨씬 다양한 메뉴가 있고 맛있는 편입니다
부정확률=8.96%, 긍정확률=88.4%

기존문장: 다른 호텔 무드 등 정도의 밝기가 여기는 가장 밝은 정도라 너무 어둡습니다
부정확률=91.35%, 긍정확률=8.73%

기존문장: 배정 받은 룸은 먼지가 가득 쌓여 있고 도저히 사용할 수 없을 거 같아서 룸 체인지 원했습니다
부정확률=80.35%, 긍정확률=20.82%

기존문장: 굿굿 휴지통은 안 비워서 일회용 팬티 그대로 있고 청소는 안 해서 칫 솔 껍질이랑 치약은 그대로 있고 화장실은 냄새나요
부정확률=85.55%, 긍정확률=15.13%

기존문장: 체크아웃을 하면서 위 

부정확률=3.39%, 긍정확률=94.55%

기존문장: 방의 침대는 정말로 멋지다 이 호텔에 예약하기 전에 다른 옵션도 살펴보십시오
부정확률=25.22%, 긍정확률=71.86%

기존문장: 위치 좋음
부정확률=3.62%, 긍정확률=94.43%

기존문장: 어떻게 그들은 더러운 수건을 접을 수 있습니까 그 얼룩을 잡으려고 하지 않는 가사에 대한 불가능 에 대해 잊어 버리십시오 연결 한 분 다음에 없는 프런트 데스크에 전화를 걸어 에 대해 불평하면 그래서 당 신의 손님이 가 작동하지 않는다고 불평 할 때 당신은 무엇을 합니까 뭐라구 끔찍한 서비스 방에 관한 유일한 좋은 점은 추운 입니다
부정확률=90.03%, 긍정확률=10.35%

기존문장: 할 수 있는 클럽 레벨 1 액 세스할 수 있습니다
부정확률=14.16%, 긍정확률=83.22%

기존문장: 깨끗하고 조용하고 좋습니다
부정확률=3.95%, 긍정확률=93.97%

기존문장: 이 불가능합니다
부정확률=90.74%, 긍정확률=9.27%

기존문장: 침대 밑에 벌레 사체나 더러운 것들이 좀 많이 보였구요
부정확률=91.4%, 긍정확률=8.75%

기존문장: 위치 룸 컨디션이 대체적으로 많이 낡았고 욕실의 청결을 많이 높여야 할 듯 합니다
부정확률=86.07%, 긍정확률=14.2%

기존문장: 주차장 이용이 상당히 불편하고 전반적인 관리가 잘 안 되고 있음
부정확률=92.72%, 긍정확률=7.29%

기존문장: 위치가 너무 좋습니다 가로수길도 가깝고 교통도 편리합니다
부정확률=3.16%, 긍정확률=94.95%

기존문장: 습하고 어둡고 묶는 내내 너무 답답함
부정확률=92.51%, 긍정확률=7.43%

기존문장: 저는 이 호텔을 5성급 놀랐어요
부정확률=34.07%, 긍정확률=65.03%

기존문장: 외관 공사가 한창이던데 객실도 공사도 필요해 보입니다
부정확률=91.53%, 긍정확률=8.86%

기존문장: 접근성 좋음
부정확률=3.58%, 긍정확률=94.47%

기존문장: 조식 가격 자체가 저렴하여 그런 듯
부정확

부정확률=86.0%, 긍정확률=14.56%

기존문장: 모텔보다 못함
부정확률=81.68%, 긍정확률=19.16%

기존문장: 조식 서비스는 만족 빵 종류 몇 개와 잼 음료 시리얼이 준비되어 있고 가짓수는 적 다 몇 개의 조식 메뉴를 선택하면 직접 조리해서 테이블로 가져다 준다 따끈따끈한 스크램블이 마음에 들어 아침만큼은 두둑히 먹었다
부정확률=6.76%, 긍정확률=90.49%

기존문장: 객실 욕실도 머리카락 등 이물질이 많고 컵도 얼룩이 많았어요
부정확률=92.32%, 긍정확률=7.75%

기존문장: 바로 앞에 편의점도 있고 직원분도 굉장히 친절했어요
부정확률=3.51%, 긍정확률=94.48%

기존문장: 화장실에선 뭔가 특유의 좋지 않은 냄새가 났고 이불은 설명하고 싶지 않지만 뭔가 이물질이 두 군데나 묻어 있었음
부정확률=90.33%, 긍정확률=10.23%

기존문장: 연말 모임으로 인기가 많아요
부정확률=10.56%, 긍정확률=86.67%

기존문장: 코로나 때문에 뷔페가 폐쇄된 것까진 이해하는데 그렇다고 주스 하나에 간에 기 별도 안 가는 작은 빵 두 덩어리는 해도 해도 너무하다 그럼 조식비를 감해 주셔야 하는데 카운터에서 그런 말도 없고 그냥 조식이 사라졌대 미리 알림도 전혀 없었다
부정확률=89.26%, 긍정확률=11.47%

기존문장: 부페는 처음에 돌잔치로 방문했다가 그 이후에는 개인적으로 자주 가는 편인데 음식이 전반적으로 가격 대비 괜찮습니다
부정확률=11.93%, 긍정확률=85.49%

기존문장: 수건이랑 옷에서 토 냄새 같은 거 나 고 좁고 가격 대비 별로입니다
부정확률=92.34%, 긍정확률=7.91%

기존문장: 기대 1도 안 하고 가면 괜찮음
부정확률=43.14%, 긍정확률=57.27%

기존문장: 신경 써 주세요
부정확률=74.02%, 긍정확률=26.81%

기존문장: 욕조에 천천히 겁 먹을 밤에는 두 개의 죽은 파리 그냥 놀아요
부정확률=85.53%, 긍정확률=15.2%

기존문장: 가격 대비 괜찮은 뷔페라고 생각합니다
부정확

부정확률=13.99%, 긍정확률=83.36%

기존문장: 방은 작고 더러운 객실 전화도 없었어요
부정확률=85.67%, 긍정확률=14.59%

기존문장: 단점 편하지 않은 직원들의 응대
부정확률=91.64%, 긍정확률=8.27%

기존문장: 좋은 가격에 잘 자 고 나왔습니다
부정확률=7.09%, 긍정확률=90.48%

기존문장: 다음 날 수영장 이용이 8시 부터 가능하다고 했으나 10시에 물 높이가 무릎 정도 였음
부정확률=90.31%, 긍정확률=10.28%

기존문장: 앞으로는 청소하시려면 미리 체크아웃을 하였는지 파악하고 하셨으면 좋겠습니다
부정확률=87.79%, 긍정확률=13.03%

기존문장: 호텔은 깨끗하고 청결감이 있으며 2명이 이용했는데 슈트 케이스를 펼쳐도 넓게 사용할 수 있었습니다
부정확률=3.83%, 긍정확률=94.01%

기존문장: 계속 불평을 했지만 전혀 들어 주 지도 않았어요
부정확률=66.14%, 긍정확률=35.9%

기존문장: 의전을 위해서 선택했던 곳인데 호텔이라고 하기엔 객실이 모텔보다 못한 듯 하고 객실에서 담배 냄새가 많이 나는데 그대로 고객에게 제공하는 것을 보면 서비스 마인드도 그렇게 좋다고 하긴 어렵지 않을까 싶은 뷔페는 참 좋은데 객실을 왜 이렇게 관리하는지
부정확률=81.8%, 긍정확률=19.65%

기존문장: 엘레베이터가 너무 좁고 노후해서 이용 시 겁이나요
부정확률=90.36%, 긍정확률=9.89%

기존문장: 위치 좋고 가격 합리적임
부정확률=3.56%, 긍정확률=94.48%

기존문장: 깨끗하고 좋았습니다
부정확률=4.61%, 긍정확률=93.19%

기존문장: 위치는 좋은데 다시 가진 않을 것 같아요
부정확률=69.05%, 긍정확률=32.11%

기존문장: 친구는 소리에 민감하지 않아 잘 잤지만 저처럼 민감한 사람이라면 좀 이용하기엔 불편했습니다
부정확률=84.6%, 긍정확률=16.41%

기존문장: 하지만 괜찮습니다
부정확률=71.87%, 긍정확률=29.67%

기존문장: 미니바 없습니다
부정확률=90.12%

부정확률=89.88%, 긍정확률=10.28%

기존문장: 이건 아니다 싶어서 한층 위로 옮겼네요
부정확률=68.85%, 긍정확률=32.65%

기존문장: 침대는 좋았어요
부정확률=9.2%, 긍정확률=88.01%

기존문장: 방 크기는 넓고 창문도 커서 답답한 느낌이 전혀 없었습니다
부정확률=4.94%, 긍정확률=92.84%

기존문장: 시티뷰의 개념이 달라졌네요 ㅎ
부정확률=56.49%, 긍정확률=44.45%

기존문장: 룸 들어서자마자 곰팡이 냄새 났어요
부정확률=91.2%, 긍정확률=9.18%

기존문장: 더 리버 사이드 호텔은 서양 기준을 아래에서 지금껏 가본 호텔 중 최악 이 한국 세계 객실과 로비 레스토랑에서 담배 냄새 에나 3 객실까지 우리는 방 냄새가 약간 견딜 수 있는 그래도 매일 두통이 서버에 3박을 하는 냄새 화장실에는 새는 방은 증거 화장실 화장지 던지 마십시오 카펫에 얼룩 방과 복도 수영장은 닫혀 머무는 내내 다른 대안을 제공 저는 절대 이 호텔을 추천합니다
부정확률=88.63%, 긍정확률=11.69%

기존문장: 하지만 시설이나 규모나 전반적으로 낙후된 호텔이 고 방음도 그닥 조식이 맛있다는 후기가 많았지만 솔직히 준비되어 있는 종류도 몇 없었고 먹을 만한 게 없었음
부정확률=59.16%, 긍정확률=42.3%

기존문장: 나름 호텔인데 느낌이 좋지 않네요
부정확률=83.64%, 긍정확률=16.92%

기존문장: 한남대교 초입에 위치해 있어 강남이나 이태원 명동 다니기는 아주 좋은 위치
부정확률=3.58%, 긍정확률=94.43%

기존문장: 4성이란 걸 믿을 수 없을 만큼 낙후된 시설 낡은 화장실 낡은 객실 정말 먹을 게 없었던 조식 ㅜㅜ 위치만 좋았던 데다가 직원들도 뭘여쭤볼 때마다 귀찮아 하시던 택시조차 안 불러주시던
부정확률=54.98%, 긍정확률=45.93%

기존문장: 오래된 호텔이라 그렇다고 할 수는 없는 침대 시트 및 욕실 청소 상태에 실망했어요
부정확률=90.92%, 긍정확률=9.25%

기존문장: 저는 예약 없이 일요일 디

부정확률=90.9%, 긍정확률=9.79%

기존문장: 에어컨이 고장 났어요
부정확률=92.56%, 긍정확률=7.64%

기존문장: 방안에 있는 창문은 어찌나 허술한지 방 한 개는 아예 사용조차 안하고 한 방에서 네 명이 같이 밤을 샜습니다
부정확률=85.66%, 긍정확률=15.09%

기존문장: 방은 나쁘지 않았다
부정확률=15.55%, 긍정확률=81.63%

기존문장: 침구가 편하고 좋았습니다
부정확률=3.6%, 긍정확률=94.4%

기존문장: 실내 불은 엄청 어두워서 낮에도 책도 못 볼 정도 새벽에 방이 엄청 울려서 일어나 보니 누가 방 잘못 찾아서 벨을 눌러서 깨버림 너무 놀람 호 텔에 벨이 왜 달려 있음
부정확률=91.53%, 긍정확률=8.85%

기존문장: 횡단보도가 멀리 있어서
부정확률=32.4%, 긍정확률=66.25%

기존문장: 커피가 잇엇으면 좋잇을 텐데 녹차만 잇어서 아쉽
부정확률=80.47%, 긍정확률=20.67%

기존문장: 객실에서 냄새나 고 가격 대비 최악이네요 ㅠㅠ
부정확률=84.29%, 긍정확률=17.06%

기존문장: 친구가 남친이랑 권태기일 때 오라며 우스 갯소리도 했네요 ㅋㅋ
부정확률=81.62%, 긍정확률=19.59%

기존문장: 부킹 측에서 피드백이 빨라서 기분이 덜 상해서 좋았어요
부정확률=9.85%, 긍정확률=87.57%

기존문장: 뷔페가 유명한 호텔답게 조식도 알차고 맛있었네요
부정확률=3.26%, 긍정확률=94.79%

기존문장: 욕실 타일이 모두 갈라져 있었고 청결 상태도 만족스럽지 못했다
부정확률=92.1%, 긍정확률=8.2%

기존문장: 일부러 후기를 적지 않는데 다음 예약하시는 분들께 꼭 알려드려야 할 것 같아 남겨요
부정확률=43.9%, 긍정확률=56.76%

기존문장: 방에 먼지 많음
부정확률=92.62%, 긍정확률=7.41%

기존문장: 조용하고 한강이 보여서 경치가 좋았습니다
부정확률=3.26%, 긍정확률=94.8%

기존문장: 더블 침대로 배정 받은 객실은 싱글 침대 두개를 붙여 논 침대였습니다
부

부정확률=3.67%, 긍정확률=94.34%

기존문장: 더블 예악을 방 없다고 트윈으로 접수되었다고 알아서 하지를 않나 역대 수많은 출장을 다니지만 최악입니다
부정확률=89.48%, 긍정확률=11.22%

기존문장: 직원도 친절하고 끝방으로 예약되어서 넓고 좋았어요
부정확률=3.5%, 긍정확률=94.52%

기존문장: 감자튀김 케찹 맛집 에어컨 끄고 잘라니까 에어컨에서 물 떨어집니다
부정확률=91.14%, 긍정확률=9.16%

기존문장: 나는 그에게 해줄 수 있는 모든 것을 던져 적어도 한 아침 조간 동일한 없다고 하더군요
부정확률=90.83%, 긍정확률=9.68%

기존문장: 종류도 다양하구요
부정확률=4.28%, 긍정확률=93.62%

기존문장: 근데 1층에 클럽 쪽에 안 좋은 냄새가 나요
부정확률=51.45%, 긍정확률=49.12%

기존문장: 욕실의 물이 줄줄 세고 퀘퀘한 냄새가 전반적으로 많이 났어요
부정확률=91.44%, 긍정확률=8.58%

기존문장: 깜짝 놀라서 뛰쳐 나왔어요
부정확률=79.21%, 긍정확률=21.84%

기존문장: 주변의 상점들이 있어 편리한 위치였다
부정확률=3.4%, 긍정확률=94.54%

기존문장: 강남 위치
부정확률=8.77%, 긍정확률=88.63%

기존문장: 수건에 무슨 건 더기 같은 거 붙어 있어서 안 썻네요
부정확률=88.98%, 긍정확률=11.34%

기존문장: 그는 노골 로 인정 하 않으셔도 아무것도 여행에 지쳐 포기하고 있는 우리의 스탠다드 더블 룸에 도착했습니다
부정확률=34.97%, 긍정확률=63.32%

기존문장: 잠자는데 조용했음
부정확률=9.69%, 긍정확률=87.64%

기존문장: 다음에 조식을 경험래 보고 싶네요
부정확률=8.46%, 긍정확률=88.85%

기존문장: 로비 엘리베이터 안내문 등은 별로 이나 남성 사우나는 매우 좋음
부정확률=44.6%, 긍정확률=54.99%

기존문장: 주차가 대박입니다
부정확률=5.53%, 긍정확률=92.05%

기존문장: 비 데는 고장 나서 작동도 안하고요 온니

In [11]:
# 모델 저장하기
torch.save(model.state_dict(), "result/MODEL_KoELECTRA-Base-v3-discriminator__1.pt")
r = pd.DataFrame(result)
r.columns = ['text','0','1']
r.to_csv('result/Result_KoELECTRA-Base-v3-discriminator__1.csv', index=False, encoding="utf-8-sig")

In [23]:
#result


In [ ]:
# 1: 92.45
# 2: 92.63
# 2: